In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
import re

## Scrapping sobre el HTML de Idealista (Manises y alrededores)

Primera pagina: https://www.idealista.com/areas/venta-viviendas/pagina-10?shape=%28%28__%7DoFb%7DiBehHu%7EBaxCs_HjhI%7D_JlbEjiCdQjuDw%7BAn_M%29%29

### Primero comprobamos que BeautifulSoup funciona con uno de los ficheros de Idealista

In [3]:
# Leer el archivo HTML
with open('../data/raw/idealista/idealista1.txt', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Analizar el contenido HTML de una forma estructurada y fácil de manipular.
soup = bs(html_content, "lxml") # Parser LXML

# Usar select para buscar el elemento <main>
main_content = soup.select('main.listing-items.core-vitals-listing-map#main-content')

# Buscar todos los <div class="item-info-container"> dentro de main_content
item_info_containers = main_content[0].select('div.item-info-container')

# Mostrar la información de cada elemento encontrado
for i, container in enumerate(item_info_containers, start=1):
    print(f"Elemento {i}:\n")
    print(container.prettify())
    print("\n" + "="*50 + "\n")

Elemento 1:

<div class="item-info-container">
 <a aria-level="2" class="item-link" href="/inmueble/84855858/" role="heading" title="Piso en calle Major, 183, El Mercado, Manises">
  Piso en calle Major, 183, El Mercado, Manises
 </a>
 <div class="price-row">
  <span class="item-price h2-simulated">
   160.000
   <span class="txt-big">
    €
   </span>
  </span>
 </div>
 <div class="item-detail-char">
  <span class="item-detail">
   3 hab.
  </span>
  <span class="item-detail">
   95 m²
  </span>
  <span class="item-detail">
   Planta 2ª exterior con ascensor
  </span>
 </div>
 <div class="item-description description">
  <p class="ellipsis">
   NEGOCIABLES
Vendo piso en muy buena zona, 81m muy bien distribuidos, 3 habitaciones 2 de ellas de matrimonio y otra un poco más pequeña, baño con bañera y un aseo de cortesía, comedor, cocina, balcón y galeria. Es un 2 con ascensor, muy bien cuidado y en perfectas condiciones.
  </p>
 </div>
 <div class="item-toolbar">
  <button class="icon-cha

## Una vez comprobado, iteramos sobre todos los ficheros.txt de la carpeta Idealista, convertimos en Dataframe y exportamos finalmente a CSV

In [4]:
# Ruta de la carpeta IDEALISTA que contiene el HTML
carpeta = "../data/raw/idealista"

# Lista para almacenar todos los datos de viviendas
datos_todas_viviendas = []

# Iterar sobre todos los archivos .txt de la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".txt"):
        # Leer el contenido del archivo
        print(archivo)
        ruta_archivo = os.path.join(carpeta, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Crear el objeto BeautifulSoup
        soup = bs(html_content, "lxml")
        
        # Buscar el main content
        main_content = soup.select('main.listing-items.core-vitals-listing-map#main-content')
        if not main_content:
            print(f"No se encontró 'main-content' en el archivo: {archivo}")
            continue
        
        # Buscar todos los <div class="item-info-container"> dentro de main_content
        item_info_containers = main_content[0].select('div.item-info-container')
        
        for container in item_info_containers:
            # Agencia
            # agencia = container.select_one('picture.logo-branding a').get('title', '').strip() if container.select_one('picture.logo-branding a') else None
            
            # Título, calle y barrio
            enlace_vivienda = container.select_one('a.item-link')
            titulo = enlace_vivienda.get_text(strip=True) if enlace_vivienda else None
            title_attr = enlace_vivienda.get('title', '') if enlace_vivienda else ''

            # Extraer calle y barrio del título
            if ',' in title_attr:  # Si hay comas en el título
                partes = [parte.strip() for parte in title_attr.split(',')]  # Dividir por comas y eliminar espacios
                calle = partes[0] if len(partes) > 0 else None  # La primera parte es la calle
                municipio = partes[-1] if len(partes) > 0 else None # El último elemento es el municipio
                barrio = partes[-2] if len(partes) > 1 else None  # El penúltimo elemento es el barrio
            else:  # Si no hay comas
                palabras = title_attr.split()  # Dividir el título por espacios
                if len(palabras) > 2:  # Si hay más de dos palabras
                    calle = ' '.join(palabras[:-2])  # Todo excepto las dos últimas palabras como calle
                    barrio = ' '.join(palabras[-2:])  # Las dos últimas palabras como barrio
                    municipio = "Manises"
                else:  # Si hay dos palabras o menos
                    calle = None  # No hay suficiente información para la calle
                    barrio = ' '.join(palabras)  # Tomar todo como barrio
                    municipio = "Manises"
                    
            # Precio
            precio = container.select_one('div.price-row span.item-price').get_text(strip=True) if container.select_one('div.price-row span.item-price') else None
            precio = re.sub(r'[^\d]', '', precio) if precio else None  # Eliminar caracteres no numéricos
            
            # Habitaciones, metros cuadrados y planta
            detalles = container.select('div.item-detail-char span.item-detail')
            habitaciones = re.sub(r'[^\d]', '', detalles[0].get_text(strip=True)) if len(detalles) > 0 else None
            metros_cuadrados = re.sub(r'[^\d]', '', detalles[1].get_text(strip=True)) if len(detalles) > 1 else None
            planta = None
            for detalle in detalles:
                texto = detalle.get_text(strip=True)
                if "hab" in texto:
                    habitaciones = re.sub(r'[^\d]', '', texto)
                elif "m²" in texto:
                    metros_cuadrados = re.sub(r'[^\d]', '', texto)
                elif "Planta" in texto:
                    planta = texto

            # Tipo Vivienda
            if 'piso' in title_attr.lower():
                tipo_vivienda = 'Piso'
            elif 'casa' in title_attr.lower() and metros_cuadrados and float(metros_cuadrados) <= 100:
                tipo_vivienda = 'Casa'
            elif ('casa' in title_attr.lower() or 'chalet' in title_attr.lower()) and metros_cuadrados and float(metros_cuadrados) > 100:
                tipo_vivienda = 'Chalet'
            else:
                tipo_vivienda = None

            # Descripción
            descripcion_elemento = container.select_one('div.item-description p.ellipsis')
            descripcion = descripcion_elemento.get_text(strip=True) if descripcion_elemento else None
            
            # Número de baños (buscar "baños" en la descripción)
            numero_baños = None
            if descripcion:
                match = re.search(r'(\d+)\s*baños?', descripcion)
                numero_baños = match.group(1) if match else None

            elif descripcion and numero_baños is None:
                # Busca expresiones como "1 baño", "2 baños", "baño individual", etc.
                match = re.search(r'(\d+)\s*(?:baño|baños?|aseos?|wc)', descripcion, re.IGNORECASE)
                numero_baños = match.group(1) if match else None
            
            # Etiquetas
            etiquetas = [tag.get_text(strip=True) for tag in container.select('div.listing-tags-container span.listing-tags')]
            
            # Garaje incluido
            garaje = "Sí" if container.select_one('div.price-row span.item-parking') else "No"

            # Almacenar los datos en un diccionario
            datos_vivienda = {
                'calle': calle,
                'barrio': barrio,
                'municipio': municipio,
                'tipo_vivienda': tipo_vivienda,
                'titulo': titulo,
                'habitaciones': habitaciones,
                'metros_cuadrados': metros_cuadrados,
                'aseos': numero_baños,
                'planta': planta,
                'descripcion': descripcion,
                'etiquetas': etiquetas,
                'garaje': garaje,
                'precio': precio
            }
            datos_todas_viviendas.append(datos_vivienda)

# Crear un DataFrame consolidado
ideal = pd.DataFrame(datos_todas_viviendas)

# Convertir las columnas a tipo numérico
ideal['precio'] = ideal['precio'].astype(str).str.replace(r'[^\d]', '', regex=True)  # Eliminar caracteres no numéricos
ideal['precio'] = pd.to_numeric(ideal['precio'], errors='coerce') 
ideal['habitaciones'] = pd.to_numeric(ideal['habitaciones'], errors='coerce')
ideal['metros_cuadrados'] = pd.to_numeric(ideal['metros_cuadrados'], errors='coerce')
ideal['aseos'] = pd.to_numeric(ideal['aseos'], errors='coerce')

print(ideal.shape)
ideal.head(2)

idealista1.txt
idealista10.txt
idealista11.txt
idealista12.txt
idealista13.txt
idealista14.txt
idealista15.txt
idealista16.txt
idealista17.txt
idealista18.txt
idealista19.txt
idealista2.txt
idealista20.txt
idealista21.txt
idealista22.txt
idealista3.txt
idealista4.txt
idealista5.txt
idealista6.txt
idealista7.txt
idealista8.txt
idealista9.txt
(644, 13)


,calle,barrio,municipio,tipo_vivienda,titulo,habitaciones,metros_cuadrados,aseos,planta,descripcion,etiquetas,garaje,precio
0,Piso en calle Major,El Mercado,Manises,Piso,"Piso en calle Major, 183, El Mercado, Manises",3,95,NaN,Planta 2ª exterior con ascensor,"NEGOCIABLES\nVendo piso en muy buena zona, 81m...",[],No,160000
1,Ático en calle Penyal d'Ifach,10,Aldaia,None,"Ático en calle Penyal d'Ifach, 10, Aldaia",3,150,NaN,Planta 3ª exterior con ascensor,None,[],No,190000


#### Nos vamos a asegurar de que no **existen duplicados**, ya que hay muchos anuncios repetidos en la plataforma. Nos basaremos en específicas columnas importantes, ya que algunas son tipo lista y dan problemas a la hora de usar ***.drop_duplicates()***

In [5]:
print(f"Antes de eliminar duplicados: {ideal.shape}")
ideal = ideal.drop_duplicates(subset=['tipo_vivienda','habitaciones', 'metros_cuadrados', 'aseos', 'planta', 'garaje', 'precio'])
print(f"Después de eliminar duplicados: {ideal.shape}")

Antes de eliminar duplicados: (644, 13)
Después de eliminar duplicados: (490, 13)


In [6]:
ideal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 490 entries, 0 to 642
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calle             490 non-null    object 
 1   barrio            490 non-null    object 
 2   municipio         490 non-null    object 
 3   tipo_vivienda     427 non-null    object 
 4   titulo            490 non-null    object 
 5   habitaciones      490 non-null    int64  
 6   metros_cuadrados  490 non-null    int64  
 7   aseos             63 non-null     float64
 8   planta            231 non-null    object 
 9   descripcion       484 non-null    object 
 10  etiquetas         490 non-null    object 
 11  garaje            490 non-null    object 
 12  precio            490 non-null    int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 53.6+ KB


In [71]:
# ideal["aseos"].value_counts()

In [72]:
# ideal["habitaciones"].value_counts()

In [73]:
# ideal.loc[ideal["precio"] == 700000, :]

In [7]:
ideal.to_csv("../data/procesado/viviendasIdealista.csv", index=True)

## Siguiente, Scrapping con el HTML de Pisos (Manises y alrededores)

Primera Pagina: https://www.pisos.com/venta/pisos-manises/

### Comprobamos que BeautifulSoup funciona con uno de los ficheros de Pisos

In [8]:
# Leer el archivo HTML
with open('../data/raw/pisos/pisos1.txt', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Analizar el contenido HTML de una forma estructurada y fácil de manipular.
soup = bs(html_content, "lxml") # Parser LXML

# Usar select para buscar el elemento <main>
main_content = soup.select('div.grid__wrapper')

# Buscar todos los <div class="item-info-container"> dentro de main_content
item_info_containers = main_content[0].select('div.ad-preview__bottom')

# Mostrar la información de cada elemento encontrado
for i, container in enumerate(item_info_containers, start=1):
    print(f"Elemento {i}:\n")
    print(container.prettify())
    print("\n" + "="*50 + "\n")

Elemento 1:

<div class="ad-preview__bottom">
 <div class="ad-preview__info">
  <div class="ad-preview__section ad-preview__section--has-textlink">
   <div class="ad-preview__inline">
    <span class="ad-preview__price">
     280.000 €
    </span>
    <span class="ad-preview__type">
     Obra nueva
    </span>
   </div>
   <span class="advertising js-textLink textLinkHipotecaParrilla" data-ga-click="textlink_venta_clic,Parrilla_hipotecas,solicita" data-ga-op-type="venta" data-text-link-id="39">
    <a class="txtlnk_39" data-url="http://pubads.g.doubleclick.net/gampad/clk?id=6835668456&amp;iu=/4900/vocento.pisos" href="#" rel="nofollow">
     Calcula tu hipoteca
    </a>
   </span>
  </div>
  <div class="ad-preview__section">
   <a class="ad-preview__title" href="/comprar/atico-la_canyada_la_canada46182-4997616700_109700/">
    Ático en Carrer 30, 32
   </a>
   <p class="p-sm ad-preview__subtitle">
    La Canyada - La Cañada (Paterna)
   </p>
  </div>
  <div class="ad-preview__section">

## Una vez comprobado, iteramos sobre todos los ficheros.txt de la carpeta Pisos, convertimos en Dataframe y exportamos finalmente a CSV

In [9]:
# Ruta de la carpeta PISOS que contiene el HTML
carpeta = "../data/raw/pisos"

# Lista para almacenar todos los datos de viviendas
datos_todas_viviendas = []

# Iterar sobre todos los archivos .txt de la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".txt"):
        # Leer el contenido del archivo
        print(archivo)
        ruta_archivo = os.path.join(carpeta, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Crear el objeto BeautifulSoup
        soup = bs(html_content, "lxml")
        
        # Buscar el contenedor principal
        main_content = soup.select('div.grid__wrapper')
        if not main_content:
            print(f"No se encontró el contenido principal en el archivo: {archivo}")
            continue
        
        # Buscar todos los <div class="ad-preview__bottom"> dentro de main_content
        item_info_containers = main_content[0].select('div.ad-preview__bottom')
        
        for container in item_info_containers:
            # Agencia (si existe información de contacto, podría deducirse como agencia)
            # agencia = container.select_one('div.contact-box')
            # agencia = agencia.get('data-is-newdevelopment', '').strip() if agencia else None
            
            # Precio
            precio = container.select_one('span.ad-preview__price').get_text(strip=True) if container.select_one('span.ad-preview__price') else None
            precio = re.sub(r'[^\d]', '', precio) if precio else None  # Eliminar caracteres no numéricos
            
            # Título, calle y barrio
            enlace_vivienda = container.select_one('a.ad-preview__title')
            titulo = enlace_vivienda.get_text(strip=True) if enlace_vivienda else None
            calle = titulo.split(',')[0].strip() if titulo else None
            
            subtitulo = container.select_one('p.ad-preview__subtitle')
            barrio = subtitulo.get_text(strip=True) if subtitulo else None
            
            # Municipio (deducido del subtítulo si está estructurado como "barrio (municipio)")
            municipio = None
            if subtitulo and '(' in subtitulo.get_text():
                municipio = re.search(r'\((.*?)\)', subtitulo.get_text()).group(1).strip()
            
            # Habitaciones, baños, metros cuadrados y planta
            detalles = container.select('p.ad-preview__char')
            habitaciones = None
            numero_baños = None
            metros_cuadrados = None
            planta = None
            for detalle in detalles:
                texto = detalle.get_text(strip=True)
                if "habs" in texto:
                    habitaciones = re.sub(r'[^\d]', '', texto)
                elif "baños" in texto:
                    numero_baños = re.sub(r'[^\d]', '', texto)
                elif "m²" in texto:
                    metros_cuadrados = re.sub(r'[^\d]', '', texto)
                elif "planta" in texto.lower():
                    planta = texto
            
            # Tipo de vivienda basado en el título y metros cuadrados
            tipo_vivienda = container.select_one('a.ad-preview__title').get_text(strip=True) if container.select_one('a.ad-preview__title') else None
            if tipo_vivienda:
                if 'piso' in tipo_vivienda.lower() or 'apartamento' in tipo_vivienda.lower():
                    tipo_vivienda = 'Piso'
                elif 'atico' in tipo_vivienda.lower() or 'ático' in tipo_vivienda.lower():
                    tipo_vivienda = 'Piso'
                elif 'duplex' in tipo_vivienda.lower() or 'dúplex' in tipo_vivienda.lower():
                    tipo_vivienda = 'Piso'
                elif 'casa' in tipo_vivienda.lower():
                    tipo_vivienda = 'Casa'
                elif 'chalet' in tipo_vivienda.lower():
                    tipo_vivienda = 'Chalet'
                else:
                    tipo_vivienda = None

            # Descripción
            descripcion_elemento = container.select_one('p.ad-preview__description')
            descripcion = descripcion_elemento.get_text(strip=True) if descripcion_elemento else None
            
            # Etiquetas (no presente explícitamente, pero se puede construir con tipo de vivienda u otros datos)
            etiquetas = [tipo_vivienda] if tipo_vivienda else []
            
            # Garaje (inferencia si está mencionado en la descripción)
            garaje = "Sí" if descripcion and "garaje" in descripcion.lower() else "No"
            
            # Almacenar los datos en un diccionario
            datos_vivienda = {
                'calle': calle,
                'barrio': barrio,
                'municipio': municipio,
                'tipo_vivienda': tipo_vivienda,
                'titulo': titulo,
                'habitaciones': habitaciones,
                'metros_cuadrados': metros_cuadrados,
                'aseos': numero_baños,
                'planta': planta,
                'descripcion': descripcion,
                'etiquetas': etiquetas,
                'garaje': garaje,
                'precio': precio
            }
            datos_todas_viviendas.append(datos_vivienda)

# Crear un DataFrame consolidado
pisos = pd.DataFrame(datos_todas_viviendas)

# Convertir las columnas a tipo numérico
pisos['precio'] = pd.to_numeric(pisos['precio'], errors='coerce')
pisos['habitaciones'] = pd.to_numeric(pisos['habitaciones'], errors='coerce')
pisos['aseos'] = pd.to_numeric(pisos['aseos'], errors='coerce')
pisos['metros_cuadrados'] = pd.to_numeric(pisos['metros_cuadrados'], errors='coerce')

# Mostrar los primeros resultados
print(pisos.shape)
pisos.head(3)

pisos1.txt
pisos10.txt
pisos11.txt
pisos12.txt
pisos2.txt
pisos3.txt
pisos4.txt
pisos5.txt
pisos6.txt
pisos7.txt
pisos8.txt
pisos9.txt
(370, 13)


,calle,barrio,municipio,tipo_vivienda,titulo,habitaciones,metros_cuadrados,aseos,planta,descripcion,etiquetas,garaje,precio
0,Ático en Carrer 30,La Canyada - La Cañada (Paterna),Paterna,Piso,"Ático en Carrer 30, 32",2.0,99,NaN,None,Tipo 16. Ático en un inmueble de obra nueva de...,[Piso],No,280000.0
1,Ático en calle de los Molinos,Santa Rita-Alborgí (Paterna),Paterna,Piso,"Ático en calle de los Molinos, s/n",3.0,218,2.0,None,Tipo D. 10º-39. Ático en un inmueble de obra n...,[Piso],Sí,376500.0
2,Piso en Carrer Miguel Hernández,Centre (Paterna),Paterna,Piso,"Piso en Carrer Miguel Hernández, 10",3.0,86,2.0,None,Vivienda 12. Piso en una finca de obra nueva d...,[Piso],No,244000.0


#### Nos vamos a asegurar de que no **existen duplicados**, ya que hay muchos anuncios repetidos en la plataforma. Nos basaremos en específicas columnas importantes, ya que algunas son tipo lista y dan problemas a la hora de usar ***.drop_duplicates()***

In [10]:
print(f"Antes de eliminar duplicados: {pisos.shape}")
pisos = pisos.drop_duplicates(subset=['tipo_vivienda','habitaciones', 'metros_cuadrados', 'aseos', 'planta', 'garaje', 'precio'])
print(f"Después de eliminar duplicados: {pisos.shape}")

Antes de eliminar duplicados: (370, 13)
Después de eliminar duplicados: (336, 13)


In [11]:
pisos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 336 entries, 0 to 369
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calle             336 non-null    object 
 1   barrio            336 non-null    object 
 2   municipio         252 non-null    object 
 3   tipo_vivienda     335 non-null    object 
 4   titulo            336 non-null    object 
 5   habitaciones      303 non-null    float64
 6   metros_cuadrados  336 non-null    int64  
 7   aseos             229 non-null    float64
 8   planta            134 non-null    object 
 9   descripcion       334 non-null    object 
 10  etiquetas         336 non-null    object 
 11  garaje            336 non-null    object 
 12  precio            333 non-null    float64
dtypes: float64(3), int64(1), object(9)
memory usage: 36.8+ KB


In [123]:
# pisos["aseos"].value_counts()

In [124]:
# pisos["habitaciones"].value_counts()

In [127]:
# pisos["tipo_vivienda"].value_counts()

In [12]:
pisos.to_csv("../data/procesado/viviendasPisos.csv", index=True)

## Siguiente, Scrapping con el HTML de Fotocasa

Primera pagina: https://www.fotocasa.es/es/comprar/viviendas/manises/todas-las-zonas/l

### Comprobamos que BeautifulSoup funciona con uno de los ficheros de Fotocasa

In [ ]:
# Leer el archivo HTML
with open('../data/raw/foto/fotocasa1.txt', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Analizar el contenido HTML de una forma estructurada y fácil de manipular.
soup = bs(html_content, "lxml") # Parser LXML
soup

### Fotocasa no nos lo pone facil para scrappear por sus etiquetas, asi que vamos a hacerlo por partes, una para ciertas etiquetas y almacenarlas en un dataframe, y otra parte para otras etiquetarlas y guardarlas en otro dataframe y finalmente unificarlos para exportar a CSV

#### Etiquetas: **'re-CardPackPremium', 're-CardPackAdvance', 're-CardPackBasic'**

In [9]:
# Ruta de la carpeta FOTOCASA que contiene el HTML
carpeta = "../data/raw/fotocasa"

# Lista para almacenar todos los datos de viviendas
datos_todas_viviendas = []

# Iterar sobre todos los archivos .txt en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".txt"):
        # Leer el contenido del archivo
        print("\n>>>>>>>>>>>>>>",archivo, "<<<<<<<<<<<<<<")
        ruta_archivo = os.path.join(carpeta, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Crear el objeto BeautifulSoup
        soup = bs(html_content, "lxml")
        
        # Buscar el contenedor principal
        main_content = soup.select_one('section.re-SearchResult')
        if not main_content:
            print(f"No se encontró el contenedor principal en el archivo: {archivo}")
            continue

        # Clases principales y sus contenedores info correspondientes
        classes = {
            're-CardPackPremium': 're-CardPackPremium-info-container',
            're-CardPackAdvance': 're-CardPackAdvance-info-container',
            're-CardPackAdvanceNewConstruction.re-CardPackAdvanceNewConstruction--newConstruction': 're-CardPackAdvanceNewConstruction-info-container',
            're-CardPackBasic': 're-CardPackBasic-info-container',
        }

        # Iterar sobre cada clase principal y su respectivo contenedor info
        for main_class, info_class in classes.items():
            # Buscar todos los artículos de la clase principal
            articles = main_content.select(f'article.{main_class}')
            if not articles:
                print(f"No se encontró el articulo {main_class}")
                continue

            print(f"-------------> Procesando clase: {main_class}, total artículos encontrados: {len(articles)}")

            # Iterar sobre cada artículo
            for article_index, article in enumerate(articles, start=1):
                # Verificar si el contenedor info existe dentro del artículo
                print(f"Artículo {article_index} de la clase {main_class}...")
                info_containers = article.select(f'a.{info_class}')
                
                if not info_containers:
                    print(f"No se encontró '{info_class}' en artículo {article_index} de la clase {main_class}")
                    continue
                
                print("Info container:", info_class)

                # Extraer información de cada contenedor info
                for container_index, info_container in enumerate(info_containers, start=1):
                    # Extraer datos relevantes
                    precio = None
                    tipo_vivienda = None
                    titulo = None
                    calle = None
                    barrio = None
                    municipio = None
                    habitaciones = None
                    aseos = None
                    metros_cuadrados = None
                    planta = None
                    descripcion = None
                    etiquetas = []
                    garaje = "No"

                    # Precio
                    precio_element = info_container.select_one('span.re-CardPrice')
                    precio = precio_element.get_text(strip=True) if precio_element else None
                    precio = re.sub(r'[^\d]', '', precio) if precio else None
                    
                    # Título, calle, barrio y municipio
                    enlace_element = info_container.get('title', '').strip()
                    titulo = enlace_element if enlace_element else None

                    # Extraer calle y barrio del título
                    if ',' in titulo:  # Si hay comas en el título
                        partes = [parte.strip() for parte in titulo.split(',')]  # Dividir por comas y eliminar espacios
                        calle = partes[0] if len(partes) > 0 else None  # La primera parte es la calle
                        barrio = partes[-1] if len(partes) > 1 else None  # El último elemento es el barrio
                    else:  # Si no hay comas
                        palabras = titulo.split()  # Dividir el título por espacios
                        if len(palabras) > 2:  # Si hay más de dos palabras
                            calle = ' '.join(palabras[:-2])  # Todo excepto las dos últimas palabras como calle
                            barrio = ' '.join(palabras[-2:])  # Las dos últimas palabras como barrio
                        else:  # Si hay dos palabras o menos
                            calle = None  # No hay suficiente información para la calle
                            barrio = ' '.join(palabras)  # Tomar todo como barrio

                    municipio = "Manises"

                    # Tipo de vivienda
                    tipo_vivienda_element = info_container.select_one('h3.re-CardHeader strong')
                    tipo_vivienda = tipo_vivienda_element.get_text(strip=True) if tipo_vivienda_element else None

                    if tipo_vivienda:
                        if "obra" in tipo_vivienda.lower():
                            tipo_vivienda = "Piso"
                        elif 'piso' in tipo_vivienda.lower() or 'apartamento' in tipo_vivienda.lower():
                            tipo_vivienda = 'Piso'
                        elif 'atico' in tipo_vivienda.lower() or 'ático' in tipo_vivienda.lower():
                            tipo_vivienda = 'Piso'
                        elif 'duplex' in tipo_vivienda.lower() or 'dúplex' in tipo_vivienda.lower():
                            tipo_vivienda = 'Piso'
                        elif 'casa o chalet' in tipo_vivienda.lower() and "el mercado" in barrio.lower() or "centro ciudad" in barrio.lower() or "el carmen" in barrio.lower():
                            tipo_vivienda = 'Casa'
                        elif 'casa o chalet' in tipo_vivienda.lower():
                            tipo_vivienda = 'Chalet'
                        elif 'finca' in tipo_vivienda.lower() and "el mercado" in barrio.lower() or "centro ciudad" in barrio.lower() or "el carmen" in barrio.lower():
                            tipo_vivienda = 'Casa'
                        elif 'finca' in tipo_vivienda.lower():
                            tipo_vivienda = 'Finca'
                        else:
                            tipo_vivienda = tipo_vivienda

                    # Habitaciones, baños, metros cuadrados, planta y etiquetas
                    detalles = info_container.select('li.re-CardFeaturesWithIcons-feature')
                    for detalle in detalles:
                        texto = detalle.get_text(strip=True)
                        print(texto)
                        if "habs" in texto:
                            habitaciones = re.sub(r'[^\d]', '', texto)
                        elif "baño" in texto:
                            aseos = re.sub(r'[^\d]', '', texto)
                        elif "m²" in texto:
                            metros_cuadrados = re.sub(r'[^\d]', '', texto)
                        elif "Planta" in texto or "planta" in texto:
                            planta = texto
                        else:
                            etiquetas.append(texto)

                    # Descripción
                    #descripcion_element = info_container.select_one('
                    # p.re-CardDescription.re-CardDescription--isTwoLines > span.re-CardDescription-text.re-CardDescription-text--isTwoLines'
                    # )
                    
                    # Modificar el selector para abarcar ambas clases
                    descripcion_element = info_container.select_one(
                        'p.re-CardDescription.re-CardDescription--isTwoLines > span.re-CardDescription-text, '
                        'p.re-CardDescription.re-CardDescription--isOnelineShort > span.re-CardDescription-text'
                        )
                    descripcion = descripcion_element.get_text(strip=True) if descripcion_element else None

                    if descripcion:
                        print("Descripcion:", descripcion[:35])

                    # Garaje (inferir si está en la descripción o etiquetas)
                    garaje = "Sí" if descripcion and "garaje" in descripcion.lower() or any("garaje" in e.lower() for e in etiquetas) else "No"

                    # Almacenar los datos en un diccionario
                    datos_vivienda = {
                        'calle': calle,
                        'barrio': barrio,
                        'municipio': municipio,
                        'tipo_vivienda': tipo_vivienda,
                        'titulo': titulo,
                        'habitaciones': habitaciones,
                        'metros_cuadrados': metros_cuadrados,
                        'aseos': aseos,
                        'planta': planta,
                        'descripcion': descripcion,
                        'etiquetas': etiquetas,
                        'garaje': garaje,
                        'precio': precio
                    }
                    # Agregar los datos a la lista
                    datos_todas_viviendas.append(datos_vivienda)
                    print("\n")

# Crear un DataFrame consolidado
fotocasa1 = pd.DataFrame(datos_todas_viviendas)

# Convertir las columnas a tipo numérico
fotocasa1['precio'] = pd.to_numeric(fotocasa1['precio'], errors='coerce')
fotocasa1['habitaciones'] = pd.to_numeric(fotocasa1['habitaciones'], errors='coerce')
fotocasa1['aseos'] = pd.to_numeric(fotocasa1['aseos'], errors='coerce')
fotocasa1['metros_cuadrados'] = pd.to_numeric(fotocasa1['metros_cuadrados'], errors='coerce')

# Mostrar los primeros resultados
print(fotocasa1.shape)
fotocasa1.head(2)


>>>>>>>>>>>>>> fotocasa1.txt <<<<<<<<<<<<<<
No se encontró el articulo re-CardPackPremium
No se encontró el articulo re-CardPackAdvance
No se encontró el articulo re-CardPackAdvanceNewConstruction.re-CardPackAdvanceNewConstruction--newConstruction
No se encontró el articulo re-CardPackBasic

>>>>>>>>>>>>>> fotocasa10.txt <<<<<<<<<<<<<<
No se encontró el articulo re-CardPackPremium
No se encontró el articulo re-CardPackAdvance
No se encontró el articulo re-CardPackAdvanceNewConstruction.re-CardPackAdvanceNewConstruction--newConstruction
No se encontró el articulo re-CardPackBasic

>>>>>>>>>>>>>> fotocasa11.txt <<<<<<<<<<<<<<
No se encontró el articulo re-CardPackPremium
No se encontró el articulo re-CardPackAdvance
No se encontró el articulo re-CardPackAdvanceNewConstruction.re-CardPackAdvanceNewConstruction--newConstruction
No se encontró el articulo re-CardPackBasic

>>>>>>>>>>>>>> fotocasa12.txt <<<<<<<<<<<<<<
No se encontró el articulo re-CardPackPremium
No se encontró el articulo 

,calle,barrio,municipio,tipo_vivienda,titulo,habitaciones,metros_cuadrados,aseos,planta,descripcion,etiquetas,garaje,precio
0,Casa o chalet en venta,en Aldaia,Manises,Chalet,Casa o chalet en venta en Aldaia,NaN,414,NaN,None,Espectacular CASA de pueblo con una ubicada PR...,[],No,230000
1,Piso en venta en Santos Justo y Pastor,El Mercado,Manises,Piso,"Piso en venta en Santos Justo y Pastor, El Mer...",2.0,53,1.0,None,Sin comisiones de agencia. Piso ubicado en la ...,[],No,60000


#### Nos vamos a asegurar de que no **existen duplicados**, ya que hay muchos anuncios repetidos en la plataforma. Nos basaremos en específicas columnas importantes, ya que algunas son tipo lista y dan problemas a la hora de usar ***.drop_duplicates()***

In [10]:
print(f"Antes de eliminar duplicados: {fotocasa1.shape}")
fotocasa1 = fotocasa1.drop_duplicates(subset=['tipo_vivienda','habitaciones', 'metros_cuadrados', 'aseos', 'planta', 'garaje', 'precio'])
print(f"Después de eliminar duplicados: {fotocasa1.shape}")

Antes de eliminar duplicados: (66, 13)
Después de eliminar duplicados: (62, 13)


In [13]:
fotocasa1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62 entries, 0 to 65
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calle             62 non-null     object 
 1   barrio            62 non-null     object 
 2   municipio         62 non-null     object 
 3   tipo_vivienda     62 non-null     object 
 4   titulo            62 non-null     object 
 5   habitaciones      56 non-null     float64
 6   metros_cuadrados  62 non-null     int64  
 7   aseos             57 non-null     float64
 8   planta            5 non-null      object 
 9   descripcion       62 non-null     object 
 10  etiquetas         62 non-null     object 
 11  garaje            62 non-null     object 
 12  precio            62 non-null     int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 6.8+ KB


In [15]:
# fotocasa1[fotocasa1["tipo_vivienda"].isna()]
# fotocasa1["barrio"].value_counts()
fotocasa1["tipo_vivienda"].value_counts()

tipo_vivienda
Piso            28
Chalet          22
Casa adosada     4
Casa             3
Planta baja      3
Finca            2
Name: count, dtype: int64

### **Segunda parte** del Webscrapping para fotocasa:

#### Etiquetas: **'re-CardPackMinimal', 're-CardPackMinimal--qualitySeal'**

In [17]:
# Ruta de la carpeta FOTOCASA que contiene el HTML
carpeta = "../data/raw/fotocasa"

# Lista para almacenar todos los datos de viviendas
datos_todas_viviendas = []

# Iterar sobre todos los archivos .txt en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith(".txt"):
        print("\n>>>>>>>>>>>>>>",archivo, "<<<<<<<<<<<<<<")
        ruta_archivo = os.path.join(carpeta, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Crear el objeto BeautifulSoup
        soup = bs(html_content, "lxml")
        
        # Buscar el contenedor principal
        main_content = soup.select_one('section.re-SearchResult')
        if not main_content:
            print(f"No se encontró el contenedor principal en el archivo: {archivo}")
            continue

        # Clases principales y sus contenedores info correspondientes
        classes = {
            're-CardPackMinimal': 're-CardPackMinimal-info-container',
            're-CardPackMinimal--qualitySeal': 're-CardPackMinimal-info-container'
        }
        
        # Iterar sobre cada clase principal y su respectivo contenedor info
        for main_class, info_class in classes.items():
            # Buscar todos los artículos de la clase principal
            articles = main_content.select(f'article.{main_class}')
            if not articles:
                print(f"No se encontró el articulo {main_class}")
                continue

            print(f"-------------> Procesando clase: {main_class}, total artículos encontrados: {len(articles)}")
            
            # Iterar sobre cada artículo
            for article_index, article in enumerate(articles, start=1):
                print(f"Artículo {article_index} de la clase {main_class}...")

                info_containers = article.select(f"div.{main_class}-info > a.{info_class}")

                if not info_containers:
                    info_containers = article.select(f"a.{info_class}")

                    if not info_containers:
                        print(f"No se encontró '{info_class}' en artículo {article_index} de la clase {main_class}")
                        continue

                print("Info container:", info_class)

                # Extraer información de cada contenedor
                for container_index, info_container in enumerate(info_containers, start=1):
                    # Extraer datos relevantes
                    precio = None
                    tipo_vivienda = None
                    titulo = None
                    calle = None
                    barrio = None
                    municipio = None
                    habitaciones = None
                    aseos = None
                    metros_cuadrados = None
                    planta = None
                    descripcion = None
                    etiquetas = []
                    garaje = "No"

                    # Precio
                    precio_element = info_container.select_one('span.re-CardPrice')
                    precio = precio_element.get_text(strip=True) if precio_element else None
                    precio = re.sub(r'[^\d]', '', precio) if precio else None

                    # Título, calle, barrio y municipio
                    enlace_element = info_container.get('title', '').strip()
                    titulo = enlace_element if enlace_element else None

                    # Extraer calle y barrio del título
                    if ',' in titulo:  # Si hay comas en el título
                        partes = [parte.strip() for parte in titulo.split(',')]  # Dividir por comas y eliminar espacios
                        calle = partes[0] if len(partes) > 0 else None  # La primera parte es la calle
                        barrio = partes[-1] if len(partes) > 1 else None  # El último elemento es el barrio
                    else:  # Si no hay comas
                        palabras = titulo.split()  # Dividir el título por espacios
                        if len(palabras) > 2:  # Si hay más de dos palabras
                            calle = ' '.join(palabras[:-2])  # Todo excepto las dos últimas palabras como calle
                            barrio = ' '.join(palabras[-2:])  # Las dos últimas palabras como barrio
                        else:  # Si hay dos palabras o menos
                            calle = None  # No hay suficiente información para la calle
                            barrio = ' '.join(palabras)  # Tomar todo como barrio

                    municipio = "Manises"

                    # Tipo de vivienda
                    tipo_vivienda_element = info_container.select_one('h3.re-CardHeader strong')
                    tipo_vivienda = tipo_vivienda_element.get_text(strip=True) if tipo_vivienda_element else None
                    
                    if tipo_vivienda:
                        if "obra" in tipo_vivienda.lower():
                            tipo_vivienda = "Piso"
                        elif 'piso' in tipo_vivienda.lower() or 'apartamento' in tipo_vivienda.lower():
                            tipo_vivienda = 'Piso'
                        elif 'atico' in tipo_vivienda.lower() or 'ático' in tipo_vivienda.lower():
                            tipo_vivienda = 'Piso'
                        elif 'duplex' in tipo_vivienda.lower() or 'dúplex' in tipo_vivienda.lower():
                            tipo_vivienda = 'Piso'
                        elif 'casa o chalet' in tipo_vivienda.lower() and "el mercado" in barrio.lower() or "centro ciudad" in barrio.lower() or "el carmen" in barrio.lower():
                            tipo_vivienda = 'Casa'
                        elif 'casa o chalet' in tipo_vivienda.lower():
                            tipo_vivienda = 'Chalet'
                        elif 'finca' in tipo_vivienda.lower() and "el mercado" in barrio.lower() or "centro ciudad" in barrio.lower() or "el carmen" in barrio.lower():
                            tipo_vivienda = 'Casa'
                        elif 'finca' in tipo_vivienda.lower():
                            tipo_vivienda = 'Finca'
                        else:
                            tipo_vivienda = tipo_vivienda

                    # Habitaciones, baños, metros cuadrados, planta y etiquetas
                    detalles = info_container.select('li.re-CardFeatures-item.re-CardFeatures-feature')
                    for detalle in detalles:
                        texto = detalle.get_text(strip=True)
                        print(texto)
                        if "habs" in texto:
                            habitaciones = re.sub(r'[^\d]', '', texto)
                        elif "baño" in texto:
                            aseos = re.sub(r'[^\d]', '', texto)
                        elif "m²" in texto:
                            metros_cuadrados = re.sub(r'[^\d]', '', texto)
                        elif "Planta" in texto or "planta" in texto:
                            planta = texto
                        else:
                            etiquetas.append(texto)

                    # Descripción
                    descripcion_element = info_container.select_one('p.re-CardDescription.re-CardDescription--isTwoLines > span.re-CardDescription-text.re-CardDescription-text--isTwoLines')

                    descripcion = descripcion_element.get_text(strip=True) if descripcion_element else None

                    if descripcion:
                        print("Descripcion:", descripcion[:35])
                    
                    # Garaje (inferir si está en la descripción o etiquetas)
                    garaje = "Sí" if descripcion and "garaje" in descripcion.lower() or any("garaje" in e.lower() for e in etiquetas) else "No"

                    # Almacenar los datos en un diccionario
                    datos_vivienda = {
                        'calle': calle,
                        'barrio': barrio,
                        'municipio': municipio,
                        'tipo_vivienda': tipo_vivienda,
                        'titulo': titulo,
                        'habitaciones': habitaciones,
                        'metros_cuadrados': metros_cuadrados,
                        'aseos': aseos,
                        'planta': planta,
                        'descripcion': descripcion,
                        'etiquetas': etiquetas,
                        'garaje': garaje,
                        'precio': precio
                    }
                    # Agregar los datos a la lista
                    datos_todas_viviendas.append(datos_vivienda)
                    print("\n")

# Crear un DataFrame consolidado
fotocasa2 = pd.DataFrame(datos_todas_viviendas)

# Convertir las columnas a tipo numérico
fotocasa2['precio'] = pd.to_numeric(fotocasa2['precio'], errors='coerce')
fotocasa2['habitaciones'] = pd.to_numeric(fotocasa2['habitaciones'], errors='coerce')
fotocasa2['aseos'] = pd.to_numeric(fotocasa2['aseos'], errors='coerce')
fotocasa2['metros_cuadrados'] = pd.to_numeric(fotocasa2['metros_cuadrados'], errors='coerce')

# Mostrar los primeros resultados
print(fotocasa2.shape)
fotocasa2.head(2)


>>>>>>>>>>>>>> fotocasa1.txt <<<<<<<<<<<<<<
-------------> Procesando clase: re-CardPackMinimal, total artículos encontrados: 30
Artículo 1 de la clase re-CardPackMinimal...
Info container: re-CardPackMinimal-info-container
2 habs.
2 baños
80 m²
Ascensor
Terraza
Descripcion: REF 2534 Se vende  magnifica vivien


Artículo 2 de la clase re-CardPackMinimal...
Info container: re-CardPackMinimal-info-container
3 habs.
2 baños
150 m²
3ª Planta
Ascensor
Descripcion: Piso  en ALDAIA - BARRIO DEL CRISTO


Artículo 3 de la clase re-CardPackMinimal...
Info container: re-CardPackMinimal-info-container
5 habs.
3 baños
280 m²
Parking
Aire acondicionado
Descripcion: CASA O CHALET EN VENTA EN CALLE CON


Artículo 4 de la clase re-CardPackMinimal...
Info container: re-CardPackMinimal-info-container
4 habs.
1 baño
96 m²
1ª Planta
Terraza
Descripcion: ¡Oportunidad única en el corazón de


Artículo 5 de la clase re-CardPackMinimal...
Info container: re-CardPackMinimal-info-container
5 habs.
3 baños
343 m

,calle,barrio,municipio,tipo_vivienda,titulo,habitaciones,metros_cuadrados,aseos,planta,descripcion,etiquetas,garaje,precio
0,Piso en venta en Quart,de Poblet,Manises,Piso,Piso en venta en Quart de Poblet,2.0,80.0,2.0,None,REF 2534 Se vende magnifica vivienda seminuev...,"[Ascensor, Terraza]",No,195000.0
1,Piso en venta,en Aldaia,Manises,Piso,Piso en venta en Aldaia,3.0,150.0,2.0,3ª Planta,"Piso en ALDAIA - BARRIO DEL CRISTO, con 150 m...",[Ascensor],No,190000.0


#### Nos vamos a asegurar de que no **existen duplicados**, ya que hay muchos anuncios repetidos en la plataforma. Nos basaremos en específicas columnas importantes, ya que algunas son tipo lista y dan problemas a la hora de usar ***.drop_duplicates()***

In [18]:
print(f"Antes de eliminar duplicados: {fotocasa2.shape}")
fotocasa2 = fotocasa2.drop_duplicates(subset=['tipo_vivienda','habitaciones', 'metros_cuadrados', 'aseos', 'planta', 'garaje', 'precio'])
print(f"Después de eliminar duplicados: {fotocasa2.shape}")

Antes de eliminar duplicados: (447, 13)
Después de eliminar duplicados: (323, 13)


In [19]:
fotocasa2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 323 entries, 0 to 413
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calle             323 non-null    object 
 1   barrio            323 non-null    object 
 2   municipio         323 non-null    object 
 3   tipo_vivienda     323 non-null    object 
 4   titulo            323 non-null    object 
 5   habitaciones      270 non-null    float64
 6   metros_cuadrados  322 non-null    float64
 7   aseos             282 non-null    float64
 8   planta            89 non-null     object 
 9   descripcion       313 non-null    object 
 10  etiquetas         323 non-null    object 
 11  garaje            323 non-null    object 
 12  precio            321 non-null    float64
dtypes: float64(4), object(9)
memory usage: 35.3+ KB


### Una vez scrappeadas las 2 partes de fotocasa, **concatenamos** los dataframes y **exportamos a CSV**

In [20]:
fotocasa = pd.concat([fotocasa1, fotocasa2], ignore_index=True)

In [21]:
fotocasa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calle             385 non-null    object 
 1   barrio            385 non-null    object 
 2   municipio         385 non-null    object 
 3   tipo_vivienda     385 non-null    object 
 4   titulo            385 non-null    object 
 5   habitaciones      326 non-null    float64
 6   metros_cuadrados  384 non-null    float64
 7   aseos             339 non-null    float64
 8   planta            94 non-null     object 
 9   descripcion       375 non-null    object 
 10  etiquetas         385 non-null    object 
 11  garaje            385 non-null    object 
 12  precio            383 non-null    float64
dtypes: float64(4), object(9)
memory usage: 39.2+ KB


In [22]:
fotocasa.to_csv("../data/procesado/viviendasFotoCasa.csv", index=True)